In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import tqdm

In [2]:
params = """remoconExpression: 
remoconDocsFound: 
remoconSelectedArticles: 
queryText: 
머신러닝
searchInResultCk: 
undefined
next: 
MainList
passwd: 
userId: 
config: 
G1111111111111111111111S111111111000000000
sortField: 
RANK
sortState: 
DESC
sortField1: 
sortState1: 
sortField2: 
sortState2: 
configChange: 
Y
expression: 
머신러닝
historyQuery:
머신러닝
numPerPage: 
30
numPageLinks: 
10
currentPage: 
1
FROM: 
BOOKMARK: 
NWBOOKMARK: 
REBOOKMARK: 
natlCD: 
beforeExpression:
prefixExpression: 
userInput: 
searchKeyword: 
searchInTrans: 
N
searchInResult: 
logFlag: 
Y
searchSaveCnt: 
0
piField: 
piValue: 
piSearchYN: 
N
leftGubnChk: 
leftHangjungChk: 
SEL_PAT: 
KPAT
merchandiseString: 
measureString: 
patternString: 
collectionValues: 
selectedLang: 
lang: 
strstat: 
TOP%7CKW
highlightKeyword: 
searchInTransCk: 
undefined
history: 
true 
"""

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
}

In [3]:
def make(params_2):
  
    lst_a = []
    lst_b = []
    lst_c = []

    params_2 = params_2.split('\n')
    for idx,i in enumerate(params_2):    
        if idx == 0:
            lst_c.append('')

        if ':' in i:
            lst_a.append(i)

            if idx == 0:
                lst_b.append('')
                continue

            if (':' in lst_c[-1]) & (':' in i):
                lst_b.append('')

        else:

            lst_b.append(i)

        lst_c.append(i)

    params = {}
    for i,j in zip(lst_a,lst_b):
        params[i.split(':')[0].strip()] = j
        
    return params
        

In [4]:
params_2 = make(params)

In [5]:
'div#divRealContent'+str(3)

'div#divRealContent3'

In [25]:
num = 1

def crawling_final(Want,num):
    

    df_test_2 = pd.DataFrame()    

    params_2['queryText'] = Want
    params_2['expression'] = Want
    params_2['historyQuery'] = Want
    params_2['currentPage'] = num 
    r = requests.post('http://kpat.kipris.or.kr/kpat/searchLogina.do?next=MainSearch',headers = headers)
    cookies_dict = {name : value for name, value in zip(r.cookies.keys(),r.cookies.values() )}

    res = requests.post('http://kpat.kipris.or.kr/kpat/resulta.do',headers= headers,
                         cookies=cookies_dict, data = params_2)

    soup = BeautifulSoup(res.content, 'html.parser')



    for num_2 in range(0,30):
        if num_2 == 0:
            df_final = soup.select('div#divRealContent'+str(num_2))
        else:
            df_final = soup.select('div#divRealContent'+str(num_2))
    

        three = [i.text.strip().split(':') for i in df_final[0].select('li.right_width')]
        two = [i.text.strip().split(':') for i in df_final[0].select('li.left_width')]       
        title = [['제목',soup.select('span.float_left input')[num_2]['title']]]
        summary = [['요약',soup.select('div.search_txt')[num_2].text.replace('\t','').replace('\n','').replace('\r','')]]


        df_test = pd.DataFrame(title + three + two + summary).set_index(0)
        

        
        lst_test = []
        
        # IPC 변환
        for i in df_test.iloc[3,0].replace('\n',' ').strip().replace('\xa0','').split('  '):
            if len(i) >= 1:
                lst_test.append(i)
            else:
                pass
            
        lst_test =  np.array(lst_test).astype('object')
        
        
        
        if len(lst_test) == 0:
            df_test.iloc[3,0] = ''
        else:
            try: 
                df_test.iloc[3,0] = lst_test
            except:
                df_test.iloc[1,0] = ''.join(a.iloc[1])
                df_test = df_test.iloc[:,[0]]
            

            
            
        lst_test = []
        
        
        
        # 피인용횟수 변환
        for i in df_test.iloc[2,0].replace('\n',' ').strip().replace('\xa0','').split('  '):
            if len(i) >= 1:
                lst_test.append(i[0])
            else:
                lst_test.append(0)
                
        df_test.iloc[2,0] = lst_test[0]        

        df_test = df_test.T
        
        if num_2 == 0:
            df_test_2 = df_test.copy()
        else:
            df_test_2 = pd.concat([df_test_2, df_test],axis=0)
            
            
    
        
    
    return df_test_2

In [30]:
a  =crawling_final('IPC=[H04L + G06N + G06Q]',10)

In [32]:
a

,제목,출원인,피인용 횟수,IPC,출원번호(일자),최종권리자,요약
1,5G 네트워크들 상에서 미디어 스트리밍 및 서비스들을 실행하기 위해 애플리케이션 제...,텐센트 아메리카 엘엘씨,0,"[H04L 67/51, H04L 67/56, H04L 67/60, H04W 4/50]",1020227034694 (2021.10.12),,방법이 제공된다. 방법은 5세대 미디어 스트리밍(5GMS) 애플리케이션 제공자에 의...
1,인터랙티브 미디어 제공 시스템 및 방법(System for providing the...,한국전자통신연구원,0,"[H04N 21/422, H04N 21/854, G06Q 50/00]",1020160061636 (2016.05.19),한국전자통신연구원,"본 발명은 인터랙티브 미디어 제공 시스템 및 방법에 관한 것으로서, 본 발명의 실시..."
1,클라우드 자원 예측 장치 및 방법(APPARATUS AND METHOD FOR PR...,한국전자통신연구원,0,"[G06F 9/50, G06F 9/455, G06N 3/08, G06N 20/00]",1020200182281 (2020.12.23),,클라우드 자원 예측 장치 및 방법이 개시된다. 본 발명의 일실시예에 따른 클라우드 ...
1,위상 양자 컴퓨터를 위한 헤테로접합의 사전 검사 및 조정(PRE-SCREENING ...,"마이크로소프트 테크놀로지 라이센싱, 엘엘씨",0,"[G06N 10/40, G06N 10/20, G06F 9/30, G06F 15/82...",1020227034304 (2021.01.20),,위상 양자 컴퓨터의 큐비트 레지스터에 사용하기 위한 반도체-초전도체 헤테로접합을 평...
1,무선 통신을 이용한 뇌파/심박도 기반의 콘텐츠 추천 시스템 및 콘텐츠 추천 방법(C...,주식회사 소소,3,"[G06F 3/01, G06Q 50/10]",1020150059735 (2015.04.28),주식회사 소소,무선 통신을 이용한 뇌파/심박도 기반의 콘텐츠 추천 시스템 및 콘텐츠 추천 방법에 ...
1,자기 스트라이프 데이터 전송 장치 및 방법(APPARATUS AND METHOD F...,삼성전자주식회사,0,"[G06K 7/08, G06Q 20/20, H04B 5/00, G07G 1/00, ...",1020160057176 (2016.05.10),,"본 발명은 데이터를 송신하기 위한 장치에 관한 것으로, 특히 자기 스트라이프(Mag..."
1,크라우드 소싱을 이용한 선박 자동 식별 정보 검증 장치 및 방법(APPARATUS ...,한국전자통신연구원,1,"[H04L 9/40, H04L 12/26]",1020150050095 (2015.04.09),한국전자통신연구원,크라우드 소싱 기법을 이용한 선박 자동 식별 정보 검증 장치 및 방법이 개시된다. ...
1,색온도 가변형 스마트 엘이디 가로등 시스템(Smart LED streetlight ...,"케이디지전자 주식회사\n, 금오공과대학교 산학협력단\n더보기",3,"[H05B 44/00, H05B 45/00, H04L 29/08, F21S 8/08...",1020160055286 (2016.05.04),"케이디지전자 주식회사 , 금오공과대학교 산학협력단","색온도 가변형 스마트 엘이디 가로등 시스템은, 외부로부터의 기후정보를 토대로 가시거..."
1,"금융투자 서비스 제공 시스템, 이를 위한 서버 및 단말기(System for pro...",주식회사 카카오페이증권,0,"[G06Q 40/06, G06Q 40/02, G06Q 40/04, G06F 3/04...",1020200173611 (2020.12.11),,"본 기술은 금융투자 서비스 제공 시스템, 이를 위한 서버 및 단말기에 관한 것이다...."
1,무선 통신 시스템에서 공간 파라미터 기반 신호 송수신 방법 및 장치(METHOD A...,엘지전자 주식회사,0,"[H04W 72/04, H04W 8/22, H04B 7/06, H04B 7/08, ...",1020227024474 (2021.03.05),,무선 통신 시스템에서 공간 파라미터에 기초하여 신호를 송수신하는 방법 및 장치가 개...


In [33]:
for num in tqdm.tqdm(range(1,10271)):
    try:
        if num == 1:
            df1 = crawling_final('IPC=[H04L + G06N + G06Q]',num)
        else:
            df1 = pd.concat([df1, crawling_final('IPC=[H04L + G06N + G06Q]',num)])
            
    except:
        print(num)
        continue
        
df2 = df1.copy()

df2 = df2.reset_index(drop = True).copy()

  3%|██▌                                                                         | 349/10270 [15:09<5:17:34,  1.92s/it]

349


  4%|██▊                                                                         | 378/10270 [16:11<5:27:56,  1.99s/it]

378


  4%|███                                                                         | 406/10270 [17:11<5:53:39,  2.15s/it]

...

 99%|████████████████████████████████████████████████████████████████████████▉ | 10118/10270 [6:33:46<05:46,  2.28s/it]

10118


100%|█████████████████████████████████████████████████████████████████████████▋| 10231/10270 [6:38:06<01:26,  2.22s/it]

10231


100%|██████████████████████████████████████████████████████████████████████████| 10270/10270 [6:39:29<00:00,  2.33s/it]


In [35]:
# df2.to_csv('Total_Patent.csv',encoding = 'utf-8-sig')

In [129]:
df2

,제목,출원인,피인용 횟수,IPC,출원번호(일자),최종권리자,요약
0,거래 주소를 생성하기 위해 N개의 키 중 M개를 사용하는 다중 승인 시스템(MULT...,"티제로 아이피, 엘엘씨",0,"[G06Q 20/38, H04L 9/14, H04L 9/08, H04L 9/06]",1020207023678 (2019.01.17),,적어도 하나의 프로세서 및 적어도 하나의 프로세서에 통신 가능하게 결합되는 적어도 ...
1,비접촉식 카드의 암호화 인증을 위한 시스템 및 방법(SYSTEMS AND METHO...,"캐피탈 원 서비시즈, 엘엘씨",0,"[H04L 9/32, H04L 9/08, H04L 9/14, G06Q 20/20, ...",1020217008095 (2019.10.01),,전송 및 수신 디바이스 간의 데이터 전송을 위한 시스템 및 방법의 예시적인 실시형태...
2,분산 원장 네트워크에서 콘텐츠의 안전한 피어 투 피어 전송을 위한 시스템 및 방법(...,프리버스 에스.엘.,0,"[H04L 67/63, G06Q 20/22, G06Q 20/06, G06Q 20/0...",1020227028088 (2020.01.16),,본 발명의 다른 측면은 DLT 네트워크에 안전하고 신뢰할 수 있으며 효율적인 방식으...
3,자동화된 장치 권한 설정 및 활성화(AUTOMATED DEVICE PROVISION...,헤드워터 리서치 엘엘씨,0,"[H04L 9/32, H04L 9/40, G06Q 30/06, G06F 21/33]",1020117019884 (2010.01.18),헤드워터 리서치 엘엘씨,서비스 정책 통신 시스템 및 방법을 위한 다양한 실시예가 개시된다. 일부 실시예에서...
4,블록체인을 기반으로 하는 공인인증서 발급시스템(Certificate issuance...,주식회사 코인플러그,4,"[G06Q 20/38, G06Q 20/40, G06Q 40/02, G06Q 20/3...",1020150109320 (2015.08.03),주식회사 코인플러그,본 발명은 공인인증서용 공개키 및 공인인증서용 개인키의 생성을 사용자가 운영하는 사...
...,...,...,...,...,...,...,...
305220,진폭및각도변조된방송신호들을복조하기위한회로,마그나폭스콘슈머일렉트로닉스캄파니,0,"[H04H 40/18, H03D 5/00, H04L 27/34, H04L 27/00]",1019810002084 (1981.06.10),,
305221,2진데이타비트블럭열을2진채널비트블럭열로코딩시키는방법(METHOD OF CODING ...,코닌클리케 필립스 엔.브이.,0,"[H04L 25/49, G11B 20/14]",1019810002526 (1981.07.11),코닌클리케 필립스 일렉트로닉스 엔.브이,
305222,데이터통신씨스템,씨비에스 코퍼레이션,0,"[H04L 25/26, H04L 12/403]",1019810002440 (1981.07.06),,
305223,데이터전송시스템(DATA TRANSMISSION SYSTEM),래칼콤석리밋티드,0,"[H04L 7/00, H04L 7/04]",1019810000127 (1981.01.19),래칼-콤석리밋티드,


In [39]:
df_missing = pd.read_csv('Missing.csv',header = None)

In [79]:
Missing = df_missing.iloc[[i for i in range(193) if i%2 == 1]]

In [80]:
Missing[0].values

array(['349', '378', '406', '452', '465', '479', '574', '771', '826',
       '878', '894', '962', '1104', '1222', '1229', '1365', '1434',
       '1491', '1678', '1682', '1708', '1833', '2215', '2271', '2351',
       '2410', '2431', '2449', '2459', '2475', '2491', '2495', '2522',
       '2547', '2601', '2712', '2811', '2892', '3045', '3138', '3261',
       '3280', '3296', '3365', '3397', '3402', '3783', '4425', '4499',
       '5418', '5421', '5431', '5446', '5483', '5614', '5615', '5616',
       '5617', '5618', '5619', '5620', '5621', '5622', '5707', '5828',
       '6054', '6248', '6491', '6581', '6759', '7042', '7227', '7400',
       '7448', '7454', '7568', '7972', '8061', '8143', '8146', '8192',
       '8211', '8225', '8406', '8432', '8546', '8772', '8871', '8990',
       '9205', '9244', '9262', '9423', '9833', '10118', '10231'],
      dtype=object)

In [113]:
num = 1

def crawling_final_missing(Want,num):
    

    df_test_2 = pd.DataFrame()    

    params_2['queryText'] = Want
    params_2['expression'] = Want
    params_2['historyQuery'] = Want
    params_2['currentPage'] = num 
    r = requests.post('http://kpat.kipris.or.kr/kpat/searchLogina.do?next=MainSearch',headers = headers)
    cookies_dict = {name : value for name, value in zip(r.cookies.keys(),r.cookies.values() )}

    res = requests.post('http://kpat.kipris.or.kr/kpat/resulta.do',headers= headers,
                         cookies=cookies_dict, data = params_2)

    soup = BeautifulSoup(res.content, 'html.parser')



    for num_2 in range(0,30):
        if num_2 == 0:
            df_final = soup.select('div#divRealContent'+str(num_2))
        else:
            df_final = soup.select('div#divRealContent'+str(num_2))
    

        three = [i.text.strip().split(':') for i in df_final[0].select('li.right_width')]
        two = [i.text.strip().split(':') for i in df_final[0].select('li.left_width')]       
        title = [['제목',soup.select('span.float_left input')[num_2]['title']]]
        summary = [['요약',soup.select('div.search_txt')[num_2].text.replace('\t','').replace('\n','').replace('\r','')]]


        df_test = pd.DataFrame(title + three + two + summary).set_index(0)
        

        
        lst_test = []
        
        # IPC 변환
        for i in df_test.iloc[3,0].replace('\n',' ').strip().replace('\xa0','').split('  '):
            if len(i) >= 1:
                lst_test.append(i)
            else:
                pass
            
        lst_test =  np.array(lst_test).astype('object')
        

        if len(lst_test) == 0:
            df_test.iloc[3,0] = ''
        else:
            
            try: 
                if type(lst_test) ==int:
                    pass
                else:
                    df_test.iloc[3,0] = ' '.join(map(str, lst_test))
            except:
                df_test.iloc[1,0] = ''.join(a.iloc[1])
                df_test = df_test.iloc[:,[0]]
            

            
            
        lst_test = []
        
        
        
        # 피인용횟수 변환
        for i in df_test.iloc[2,0].replace('\n',' ').strip().replace('\xa0','').split('  '):
            if len(i) >= 1:
                lst_test.append(i[0])
            else:
                lst_test.append(0)
                
        df_test.iloc[2,0] = lst_test[0]        

        df_test = df_test.T
        
        if num_2 == 0:
            df_test_2 = df_test.copy()
        else:
            df_test_2 = pd.concat([df_test_2, df_test],axis=0)
            
            
    
        
    
    return df_test_2

In [102]:
Missing_value = [int(i) for i in Missing[0].values]

In [105]:
Missing_value[0]

349

In [122]:
Missing_value[]

378

In [125]:



for num in tqdm.tqdm(Missing_value):
    try:
        if num == 349:
            df_missing= crawling_final_missing('IPC=[H04L + G06N + G06Q]',num)
        else:
            df_missing = pd.concat([df_missing, crawling_final_missing('IPC=[H04L + G06N + G06Q]',num)])
            
    except:
        print(num)
        continue

        
df_missing_2 = df_missing.copy()
df_missing_2 = df_missing_2.reset_index(drop = True).copy()

100%|██████████████████████████████████████████████████████████████████████████████████| 96/96 [03:48<00:00,  2.38s/it]


In [127]:
df_missing_2

,제목,출원인,피인용 횟수,IPC,출원번호(일자),최종권리자,요약
0,다이어트 시스템(Diet system),"박준흥\n, 이윤성\n더보기",0,G06Q 50/22 H04L 51/00 G06T 13/40 G06N 3/00 G06...,1020210086789 (2021.07.02),"박준흥 , 이윤성","본 발명은, 설문내역을 불러오고, 각 항목별 응답을 입력받아 관리서버로 전송하는 단..."
1,차량 공유를 위한 이동통신 단말의 디지털 키를 관리하는 방법 및 이를 이용한 키 서...,현대오토에버 주식회사,2,G06Q 50/30 G07C 9/00 H04L 65/40 H04L 9/40,1020200064566 (2020.05.28),현대오토에버 주식회사,본 발명은 차량 공유를 위한 이동통신 단말의 디지털 키(Digital Key)를 관...
2,허가 기반 리소스 및 서비스 발견(PERMISSION BASED RESOURCE A...,"콘비다 와이어리스, 엘엘씨",0,H04L 9/40 H04L 65/40 H04W 12/08 H04W 4/70,1020217033913 (2015.11.13),,현재의 발견 메커니즘들은 예로서 주어진 등록자와 연계된 허가들에 관한 능력들 같은 ...
3,메모리 시스템에 구현된 메시지 전달 인터페이스를 통한 메모리 액세스 통신(MEMOR...,"마이크론 테크놀로지, 인크.",0,G06F 13/16 G06N 3/02,1020207025351 (2019.01.18),,"메모리 시스템으로서, 복수의 메모리 컴포넌트들 및 메모리 컴포넌트들에 데이터를 저장..."
4,안료 관리 시스템(PIGMENTS MANAGEMENT SYSTEM),오키드인더스트리 주식회사,0,G06Q 10/08 G06Q 10/06 B65G 1/02 G06K 7/10 G08B...,1020210085379 (2021.06.30),오키드인더스트리 주식회사,창고 프레임부 및 판넬부에 의해 밀폐되는 내부 공간을 형성하여 안료 포장 박스의 보...
...,...,...,...,...,...,...,...
3002,브리지의어드레스매칭판별장치,삼성전자주식회사,0,H04L 12/743,1019920002967 (1992.02.26),,제1네트워크에서 전송되는 데이타 패킷에 포함된 목적지 어드레스를 추출하는 제1제어장...
3003,최대가능성시퀀스메트릭계산기(MAXIMUM LIKELIHOOD SEQUENCE MET...,"실리콘 시스템즈, 인크. (대표자",1,H04L 25/03 H03M 13/41 G11B 20/18,1019920003979 (1992.03.11),"실리콘 시스템즈, 인크. (대표자",최대 가능성 시퀀스 계산기(maximum likelihood sequence met...
3004,None,윌리엄 이. 벤더쉬),None,None,None,윌리엄 이. 벤더쉬),None
3005,자동차량추적장치의데이타클럭복조회로(DATA CLOCK DEMODULATING CIR...,삼성전자주식회사,0,H04L 7/027 H04L 7/00,1019920003380 (1992.02.29),삼성전자주식회사,이 발명에 의한 자동차량 추적장치에 사용되는 데이타 클럭 복조회로는 데이타를 수신하...


In [128]:
# df_missing_2.to_csv('Total_Patent_missing.csv',encoding = 'utf-8-sig')

In [19]:
df2.shape

(8527, 7)

In [20]:
df2.drop_duplicates('출원번호(일자) ')

,제목,출원인,피인용 횟수,IPC,출원번호(일자),최종권리자,요약
0,상온유통이 가능한 된장소스 및 그 제조방법(SHELF-STABLE FERMENTED...,씨제이제일제당 (주),7,"[A23L 23/00, A23L 11/50]",1020120129735 (2012.11.15),씨제이제일제당 (주),본 발명은 한국 전통 조미료인 된장을 이용한 된장소스 및 그 제조방법에 관한 것이며...
1,블록체인 기반의 B2B 쿠폰 마케팅 시스템을 이용한 B2C 온라인 휴대폰 유통 시스...,주식회사 아이정보통신,0,"[G06Q 30/06, G06Q 30/02, H04L 9/00, H04L 67/10...",1020220077989 (2022.06.27),주식회사 아이정보통신,"본 발명은 블록체인 기술을 이용하여 쿠폰 발행 및 지급 내역의 위변조를 방지하고, ..."
2,식품 유통 통합관리솔루션 제공 시스템(Food Distribution Integra...,주식회사 케이에프,0,"[G06Q 10/06, G06Q 10/10, G06Q 50/32, G06Q 50/18]",1020200118966 (2020.09.16),주식회사 케이에프,"본 발명은 식품 유통 통합관리솔루션 제공 시스템에 관한 것으로, 그 구성은 식품회사..."
3,알림정보를 제공하는 유통기한 관리시스템(expiration date manageme...,주식회사 니즈,0,"[G06Q 10/08, G06Q 10/10, G08B 21/18, G06K 19/0...",1020220044712 (2022.04.11),주식회사 니즈,본 발명은 알림정보를 제공하는 유통기한 관리시스템에 관한 것이다.
4,축산물 유통관리 방법(Livestock product distribution man...,정성용,0,"[G06Q 30/06, G06Q 50/02, G06Q 30/02]",1020210056482 (2021.04.30),,본 발명은 축산물 유통관리 방법에 관한 것이다. 축산물 유통관리 방법은 소정의 축산...
...,...,...,...,...,...,...,...
8522,항균탈지면(Antimicrobial Gauze),김택영,1,"[A61F 13/02, A61F 13/00, A61L 15/18, A61L 15/4...",2020060021923 (2006.08.16),김택영,본 고안은 의료용 거즈(Gauze)로 사용하기 위한 항균탈지면과 그 제조방법에 관한...
8523,열선을 이용한 랩 컷팅 장치(Wrap cutting device),유수준,1,"[B65H 35/04, B65H 16/00]",2020060021560 (2006.08.11),유수준,본 고안은 열선을 이용한 포장용 랩 컷팅 장치에 관한 것이다. 이를 위해 본 고안은...
8524,청바지(blue pants),문호열,1,"[A41D 27/08, A41D 1/06]",2020060021483 (2006.08.10),문호열,"본 고안은 청바지에 관한 것이다. 본 고안은 앞판과 뒷판으로 이루어지고, 앞판과 뒷..."
8525,보조가스포켓을 가진 팽창가능한 가스백(INFLATABLE GAS BAG HAVING...,"리드팩 인더스트리얼\n, 라이 흥 핀\n더보기",0,"[B65D 81/03, B65D 81/02]",2020060020810 (2006.08.03),"리드팩 인더스트리얼 , 라이 흥 핀",본 고안은 보조가스포켓이 달린 팽창가능한 가스백인데 이는 복수개의 가스포켓유닛로 구...


In [21]:
# df2.to_csv('유통.csv',encoding = 'utf-8-sig')